In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import pandas as pd
import time
import datetime
import re
import sqlite3 as lite

# Data Collection and Integration

In [2]:
url = 'http://web.mta.info/developers/turnstile.html'
chromedriver = "/Users/Miya/Downloads/chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get(url)
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')


# extract data link
pattern = re.compile('^data.')
data_list =  []
for link in soup.find_all('a', href=True):
    if re.match(pattern,link['href']):
        data_list.append('http://web.mta.info/developers/'+link['href'])

In [3]:
#check: data should range from 05/05/2010 to 04/22/2017
print(data_list[0])
print(data_list[-1][-10:-4])

http://web.mta.info/developers/data/nyct/turnstile/turnstile_170422.txt
100505


Two data schemas for before and after 10/18/14. Therefore, we needs to locate from where schema changes

In [5]:
for i, v in enumerate(data_list):
    if v[-10:] == '141011.txt':
        print(i)

132


In [3]:
data_list_prior = data_list[132:]
data_list_post = data_list[:132]

let us create two tables to store the data 

In [4]:
conPost = lite.connect('post.db')
conPre = lite.connect('pre.db')

**A.** Collect and integrate data ** AFTER** 10/18/14 

In [ ]:
record_cnt = 0  
for link in data_list_post:
    data = pd.read_table(link, sep=',')
    print('%s:%s rows %s columns' % (link[-10:-4],data.shape[0], data.shape[1])) #printing out values makes me feel safe....
    record_cnt += data.shape[0]
    data.to_sql(name='post', con=conPost, flavor='sqlite', if_exists='append')

Check we don't miss anything

In [23]:
with conPost:
    cur = conPost.cursor()
    cur.execute("""select count(*) from post""")
    cnt = cur.fetchall()
    print(cnt)
    print(record_cnt)

[(25304899,)]
25304899


**B.** Collect and integrate data ** BEFORE** 10/18/14 

In [ ]:
record_cnt = 0 
col_names = ["C/A","UNIT","SCP","DATE","TIME","DESC","ENTRIES","EXITS"]

for link in data_list_prior:
    for i in range(3,39,5):
        data = pd.read_table(link, sep=',',header = None,usecols = [0,1,2,i,i+1,i+2,i+3,i+4],names = col_names)
        print('%s:%s rows %s columns' % (link[-10:-4],data.shape[0], data.shape[1]))
        record_cnt += data.shape[0]
        data.to_sql(name='prePart', con=conPre, flavor='sqlite', if_exists='append')

with conPre:
    cur = conPre.cursor()
    cur.execute("""select count(*) from prePart""")
    cnt = cur.fetchall()  

141018:184960 rows 8 columns


C:\Users\Miya\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


141018:184960 rows 8 columns
141018:184960 rows 8 columns
141018:184960 rows 8 columns
141018:184960 rows 8 columns
141018:184960 rows 8 columns
141018:184960 rows 8 columns
141018:184960 rows 8 columns
141011:29797 rows 8 columns
141011:29797 rows 8 columns
141011:29797 rows 8 columns
141011:29797 rows 8 columns
141011:29797 rows 8 columns
141011:29797 rows 8 columns
141011:29797 rows 8 columns
141011:29797 rows 8 columns
141004:30125 rows 8 columns
141004:30125 rows 8 columns
141004:30125 rows 8 columns
141004:30125 rows 8 columns
141004:30125 rows 8 columns
141004:30125 rows 8 columns
141004:30125 rows 8 columns
141004:30125 rows 8 columns
140927:29972 rows 8 columns
140927:29972 rows 8 columns
140927:29972 rows 8 columns
140927:29972 rows 8 columns
140927:29972 rows 8 columns
140927:29972 rows 8 columns
140927:29972 rows 8 columns
140927:29972 rows 8 columns
140920:29628 rows 8 columns
140920:29628 rows 8 columns
140920:29628 rows 8 columns
140920:29628 rows 8 columns
140920:29628 

In [ ]:
print(cnt)
print(record_cnt)

**C.** Collect data about Remote Unit/Control Area/Station Name Key

In [ ]:
col_names = ["UNIT",'C/A','STATION','LINENAME','DIVISION']
remote = pd.read_excel("http://web.mta.info/developers/resources/nyct/turnstile/Remote-Booth-Station.xls", names=col_names).drop_duplicates(['UNIT','C/A'])
remote.to_sql(name='remote', con=conPre, flavor='sqlite', if_exists='replace')

with conPre:
    cur = con.cursor()
    cur.execute("""select count(*) from remote""")
    cnt = cur.fetchall()
    print(cnt)

**D.** Complete data before 10/18/14 by joining two datasets. *** PrimaryKey**: C/A(BOOTH) and UNIT

In [ ]:
with conPre:
    cur = con.cursor()
    cur.execute("""Create table prior as select * from prePart left join remote on prePart.UNIT = remote.UNIT AND prePart.[C/A] = remote.[C/A]""")

In [ ]:
## check
with conPre:
    cur = con.cursor()
    cur.execute("""select count(*) from prior """)
    join_cnt = cur.fetchall()
    print(join_cnt)

Now we have two databases: post.db and pre.db

1. **post.db** has **one** table called **post**, which stores complete information about data after 10/18/14;

2. **pre.db** has **three** tables called **prePart,remote and prior**;

3. Table **prior** stores complete information about data before 10/18/14;

Let's see schemas of two tables:

In [ ]:
with conPre:
    cur = conPre.cursor()
    cur.execute("""select sql from sqlite_master where type = 'table' and name = 'prior'""")
    desc = cur.fetchall()
    print(desc)

In [ ]:
with conPost:
    cur = conPost.cursor()
    cur.execute("""select sql from sqlite_master where type = 'table' and name = 'post'""")
    desc = cur.fetchall()
    print(desc)

In [ ]:
with conPre:
    cur = conPre.cursor()
    cur.execute("""select sql from sqlite_master where type = 'table' and name = 'prePart'""")
    desc = cur.fetchall()
    print(desc)

# Data Analysis

#### Which station has the most number of units?
    
   **Answer：** 23 ST  

In [ ]:
start = datetime.datetime.now()
with conPost:
    cur = conPost.cursor()
    cur.execute("""select station, unit from post """)
    postD = cur.fetchall()
with conPre:
    cur = conPre.cursor()
    cur.execute("""select station, unit from prior """)
    priorD = cur.fetchall()
postD_df = pd.DataFrame(postD,columns = ['station','unit'])
priorD_df = pd.DataFrame(priorD,columns = ['station','unit'])
print(pd.concat([postD_df,priorD_df]).drop_duplicates(['station','unit']).groupby('station').count().sort_values(by = 'unit',ascending = False).head(1))
end = datetime.datetime.now()
print('Spend %s seconds'%(end-start).seconds)

#### What is the total number of entries & exits across the subway system for August 1, 2013?

   **Answer：** 4483011530

In [ ]:
start = datetime.datetime.now()

with conPre:
    cur = conPre.cursor()
    cur.execute("""select DATE,sum(ENTRIES) AS s_ENTRIES, sum(EXITS) AS s_EXITS from prior \
    group by DATE HAVING DATE in ('08-01-13','07-31-13)""")
    sumEE = cur.fetchall()
sumEE_df = pd.DataFrame(sumEE,columns = ["DATE",'s_ENTRIES','s_EXITS'])
sumEE_df['TOTAL'] = sumEE_df['s_ENTRIES']+sumEE_df['s_EXITS']
end = datetime.datetime.now()
print(sumEE_df['TOTAL'].diff().values[1])
print('Spend %s seconds'%(end-start).seconds)

#### Let’s define the busy-ness as sum of entry & exit count. What station was the busiest on August 1, 2013? What turnstile was the busiest on that date?
*A tunstile is identified by SCP and C/A*

In [67]:
start = datetime.datetime.now()
with conPre:
    cur = conPre.cursor()
    cur.execute("""select DATE, STATION,sum(ENTRIES) AS s_ENTRIES, sum(EXITS) AS s_EXITS from prior \
    group by DATE,STATION Having DATE IN ('08-01-13','07-31-13') """)
    S_busiest = cur.fetchall()
S_busiest_df = pd.DataFrame(S_busiest,columns = ['DATE','STATION','s_ENTRIES','s_EXITS'])
S_busiest_df['TOTAL'] = S_busiest_df['s_ENTRIES']+S_busiest_df['s_EXITS']
end = datetime.datetime.now()
print(busiest)
print('Spend %s seconds'%(end-start).seconds)

[('183 ST', 1650277551)]
Spend 12 seconds


In [ ]:
start = datetime.datetime.now()
with conPre:
    cur = conPre.cursor()
    cur.execute("""select DATE, [C/A],SCP, sum(ENTRIES) AS s_ENTRIES, sum(EXITS) AS s_EXITS from prior \
    group by DATE,[C/A],SCP Having DATE IN ('08-01-13','07-31-13') """)
    T_busiest = cur.fetchall()
T_busiest_df = pd.DataFrame(T_busiest_df,columns = ['DATE','STATION','s_ENTRIES','s_EXITS'])
T_busiest_df['TOTAL'] = T_busiest_df['s_ENTRIES']+T_busiest_df['s_EXITS']
end = datetime.datetime.now()
print(busiest)
print('Spend %s seconds'%(end-start).seconds)

In [70]:
S_busiest_df.groupby(['STATION','DATE']).sum()

s_ENTRIES    s_EXITS
STATION         DATE                          
1 AVE           07-31-13   52762934   82996760
                08-01-13   22083142   34760337
103 ST          07-31-13  135410587  111380754
                08-01-13  103306298   81749607
103 ST-CORONA   07-31-13   49816105   47235730
                08-01-13   44904932   32446295
104 ST          07-31-13   10589602   10315641
                08-01-13   10592092   10317591
110 ST          07-31-13   26044726   23387267
                08-01-13    8431239   16329837
110 ST-CATHEDRL 07-31-13   23801306   10048896
                08-01-13   34620416   14560831
110 ST-CPN      07-31-13   92827499    4369526
                08-01-13    8361156    3836178
111 ST          07-31-13   54120678   39258056
                08-01-13   49006065   32653568
116 ST          07-31-13   61302642   56384694
                08-01-13   79618261   65425016
116 ST-COLUMBIA 08-01-13   44273225   15985795
121 ST          07-31-13    5893685    5312279
                08-01-13    5896229    5314162
125 ST          07-31-13  173220244  126152516
                08-01-13  161860354  123100664
135 ST          07-31-13   42525920   24562640
                08-01-13   41225982   23127426
137 ST-CITY COL 07-31-13  644772277 -838894753
                08-01-13  644788380 -838881408
138 ST-3 AVE    07-31-13   16261277   15587775
                08-01-13   16271308   15594513
138 ST-GR CONC  07-31-13    7547104    7076415
...                             ...        ...
W 4 ST-WASH SQ  07-31-13   81696650   83613080
                08-01-13  251973192  105992101
W 8 ST-AQUARIUM 07-31-13  111566678   11077857
                08-01-13    3988897    1667519
WAKEFIELD-241   07-31-13   11880731   12439060
                08-01-13   11888100   12445179
WALL ST         07-31-13  619500738  181683271
                08-01-13  547518680  171464404
WASHINGTON-36 A 07-31-13   23065738   21571300
                08-01-13   23074754   21581372
WESTCHESTER SQ  07-31-13   18721412   17769826
                08-01-13   16932851   15790462
WHITEHALL ST    07-31-13   42291216  497272487
                08-01-13   42273285  497224217
WHITLOCK AVE    07-31-13    4399979    3095211
                08-01-13    4401595    3097122
WILSON AVE      07-31-13   11022960    9829430
                08-01-13   11027728    9833425
WINTHROP ST     07-31-13   25770533    4204525
                08-01-13   25780890    4206959
WOODHAVEN BLVD  07-31-13   58334882   40662054
                08-01-13   36465141   21807342
WOODLAWN ROAD   07-31-13   21089938    9404280
                08-01-13   21098236    9406214
WORLD TRADE CTR 07-31-13   27226303   14478551
                08-01-13   19889362   10428685
YORK ST         07-31-13   24439231   23203410
                08-01-13   24449383   23212927
ZEREGA AVE      07-31-13   10384828    6805419
                08-01-13    8477437    2825941

[754 rows x 2 columns]

#### What stations have seen the most usage growth/decline in 2013

In [ ]:
start = datetime.datetime.now()
with conPre:
    cur = conPre.cursor()
    cur.execute("""select DATE, STATION, sum(ENTRIES) AS s_ENTRIES, sum(EXITS) AS s_EXITS  from prior \
    group by STATION,DATE Having DATE LIKE '%13' """)
    Trend = cur.fetchall()
Trend_df = pd.DataFrame(Trend_df,columns = ['DATE','STATION','s_ENTRIES','s_EXITS'])
Trend_df['TOTAL'] = Trend_df['s_ENTRIES']+Trend_df['s_EXITS']
end = datetime.datetime.now()
print(busiest)
print('Spend %s seconds'%(end-start).seconds)

In [38]:
pd.read_table('http://web.mta.info/developers/data/nyct/turnstile/turnstile_100505.txt', sep=',')

,A002,R051,02-00-00,04-17-10,00:00:00,REGULAR,002704717,000928793,04-17-10.1,04:00:00,...,04-18-10,00:00:00.1,REGULAR.6,002705426,000928987,04-18-10.1,04:00:00.1,REGULAR.7,002705426.1,000928987
0,A002,R051,02-00-00,04-18-10,08:00:00,REGULAR,2705436,929002,04-18-10,12:00:00,...,04-19-10,08:00:00,REGULAR,2705937.0,929229.0,04-19-10,12:00:00,REGULAR,2706063.0,929533.0
1,A002,R051,02-00-00,04-19-10,16:00:00,REGULAR,2706335,929588,04-19-10,20:00:00,...,04-20-10,08:00:00,DOOR,2707246.0,929705.0,04-20-10,12:00:00,OPEN,2707385.0,930006.0
2,A002,R051,02-00-00,04-20-10,16:00:00,REGULAR,2707710,930067,04-20-10,20:00:00,...,04-21-10,16:00:00,REGULAR,2709171.0,930618.0,04-21-10,18:40:52,DOOR,2709868.0,930669.0
3,A002,R051,02-00-00,04-21-10,20:00:00,REGULAR,2710045,930698,04-22-10,00:00:00,...,04-22-10,20:00:00,REGULAR,2711562.0,931222.0,04-23-10,00:00:00,REGULAR,2711702.0,931240.0
4,A002,R051,02-00-00,04-23-10,04:00:00,REGULAR,2711708,931241,04-23-10,08:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A002,R051,02-00-01,04-17-10,00:00:00,REGULAR,2697632,566683,04-17-10,04:00:00,...,04-18-10,00:00:00,REGULAR,2698303.0,566807.0,04-18-10,04:00:00,REGULAR,2698303.0,566807.0
6,A002,R051,02-00-01,04-18-10,08:00:00,REGULAR,2698309,566813,04-18-10,12:00:00,...,04-19-10,08:00:00,REGULAR,2698748.0,566940.0,04-19-10,12:00:00,REGULAR,2698894.0,567082.0
7,A002,R051,02-00-01,04-19-10,16:00:00,REGULAR,2699141,567118,04-19-10,20:00:00,...,04-20-10,16:00:00,REGULAR,2700428.0,567367.0,04-20-10,20:00:00,REGULAR,2701157.0,567414.0
8,A002,R051,02-00-01,04-21-10,00:00:00,REGULAR,2701241,567431,04-21-10,04:00:00,...,04-22-10,00:00:00,REGULAR,2702581.0,567728.0,04-22-10,04:00:00,REGULAR,2702585.0,567729.0
9,A002,R051,02-00-01,04-22-10,08:00:00,REGULAR,2702626,567778,04-22-10,12:00:00,...,04-23-10,08:00:00,REGULAR,2703984.0,568077.0,04-23-10,12:00:00,REGULAR,2704125.0,568234.0


In [47]:
col_names = ["C/A","UNIT","SCP","DATE","TIME","DESC","ENTRIES","EXITS"]
pd.read_table('http://web.mta.info/developers/data/nyct/turnstile/turnstile_100505.txt', sep=',',
              header = None,usecols = [0,1,2,3,4,5,6,7], names = col_names).groupby(['C/A', 'SCP','DATE','TIME']).sum()['ENTRIES'].sort_values()

C/A    SCP       DATE      TIME    
R318   00-05-01  04-18-10  09:00:00           0
N187   00-05-01  04-21-10  01:00:00           0
                 04-19-10  17:00:00           0
                 04-18-10  09:00:00           0
                 04-17-10  01:00:00           0
R121   01-05-01  04-19-10  05:00:00           0
                 04-18-10  09:00:00           0
A046   00-05-01  04-17-10  01:00:00           0
N187   00-05-01  04-22-10  09:00:00           0
A046   00-05-01  04-18-10  09:00:00           0
                 04-19-10  13:00:00           0
                 04-20-10  21:00:00           0
                 04-22-10  01:00:00           0
                 04-23-10  09:00:00           0
R204A  03-05-01  04-17-10  00:00:00           0
N604   00-05-01  04-23-10  17:00:00           0
                 04-22-10  09:00:00           0
R121   01-05-01  04-17-10  01:00:00           0
N187   00-05-01  04-23-10  13:00:00           0
N001   01-05-01  04-23-10  17:00:00           0
    